# Add trial labels

Brief 1-2 sentence description of notebook.

In [1]:
import os
import glob
import git
import sys


In [2]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter


In [3]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [4]:
git_root

'/nancy/projects/reward_competition_extention'

In [5]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [6]:
# sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

## Functions

## Inputs & Data

Explanation of each input and where it comes from.

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [7]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [8]:
FULL_LFP_TRACES_PKL = "{}_03_spectral_trial_labels.pkl".format(OUTPUT_PREFIX)

## Processing

In [9]:
subject_start_stop_frames = pd.read_excel("rce_per_subject_start_stop_video_frame.xlsx")
subject_start_stop_frames = subject_start_stop_frames.dropna(subset=["file_path"])

In [10]:
subject_start_stop_frames["sleap_name"] = subject_start_stop_frames["file_path"].apply(lambda x: os.path.basename(x))
subject_start_stop_frames["video_name"] = subject_start_stop_frames["file_path"].apply(lambda x: ".".join(os.path.basename(x).split(".")[:2]))
subject_start_stop_frames["start_frame"] = subject_start_stop_frames["start_frame"].astype(int)
subject_start_stop_frames["stop_frame"] = subject_start_stop_frames["stop_frame"].astype(int)

In [11]:
subject_start_stop_frames = subject_start_stop_frames.drop(columns=["file_path", "notes", "in_video_subjects"], errors="ignore")

In [12]:
subject_start_stop_frames["video_name"].unique()

array(['20221214_125409_om_and_comp_6_1_and_6_3.1',
       '20221215_145401_comp_amd_om_6_1_and_6_3.1',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.2',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.1',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.2',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.1',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.2',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.3',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.1',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.2',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.1',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.3',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.1',
       '

In [13]:
subject_start_stop_frames.head()

,start_frame,stop_frame,tracked_subject,sleap_name,video_name
0,1,25000,6.3,20221214_125409_om_and_comp_6_1_and_6_3.1.fixe...,20221214_125409_om_and_comp_6_1_and_6_3.1
1,27500,73600,6.1_6.3,20221214_125409_om_and_comp_6_1_and_6_3.1.fixe...,20221214_125409_om_and_comp_6_1_and_6_3.1
2,51500,76454,6.3,20221215_145401_comp_amd_om_6_1_and_6_3.1.fixe...,20221215_145401_comp_amd_om_6_1_and_6_3.1
3,1,48500,6.1_6.3,20221215_145401_comp_amd_om_6_1_and_6_3.1.fixe...,20221215_145401_comp_amd_om_6_1_and_6_3.1
4,32700,68257,1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...


In [14]:
subject_start_stop_frames["current_subject"] = subject_start_stop_frames["tracked_subject"].apply(lambda x: str(x).split("_"))

In [15]:
subject_start_stop_frames = subject_start_stop_frames.explode("current_subject")

In [16]:
subject_start_stop_frames.head()

,start_frame,stop_frame,tracked_subject,sleap_name,video_name,current_subject
0,1,25000,6.3,20221214_125409_om_and_comp_6_1_and_6_3.1.fixe...,20221214_125409_om_and_comp_6_1_and_6_3.1,6.3
1,27500,73600,6.1_6.3,20221214_125409_om_and_comp_6_1_and_6_3.1.fixe...,20221214_125409_om_and_comp_6_1_and_6_3.1,6.1
1,27500,73600,6.1_6.3,20221214_125409_om_and_comp_6_1_and_6_3.1.fixe...,20221214_125409_om_and_comp_6_1_and_6_3.1,6.3
2,51500,76454,6.3,20221215_145401_comp_amd_om_6_1_and_6_3.1.fixe...,20221215_145401_comp_amd_om_6_1_and_6_3.1,6.3
3,1,48500,6.1_6.3,20221215_145401_comp_amd_om_6_1_and_6_3.1.fixe...,20221215_145401_comp_amd_om_6_1_and_6_3.1,6.1


In [21]:
TRIAL_LABELS_DF = pd.read_excel("./rce_pilot_2_per_video_trial_labels.xlsx")
TRIAL_LABELS_DF = TRIAL_LABELS_DF.rename(columns={col: col.strip().lower() for col in TRIAL_LABELS_DF.columns})
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["condition"])
TRIAL_LABELS_DF["tone_start_timestamp"] = TRIAL_LABELS_DF["tone_start_timestamp"].astype(np.int64)
TRIAL_LABELS_DF["tone_stop_timestamp"] = TRIAL_LABELS_DF["tone_stop_timestamp"].astype(np.int64)
TRIAL_LABELS_DF["video_name"] = TRIAL_LABELS_DF["video_name"].apply(lambda x: ".".join(os.path.basename(x).split(".")[:2]))

TRIAL_LABELS_DF["tone_start_frame"] = TRIAL_LABELS_DF["tone_start_frame"].astype(int)
TRIAL_LABELS_DF["reward_start_frame"] = TRIAL_LABELS_DF["reward_start_frame"].astype(int)
TRIAL_LABELS_DF["tone_stop_frame"] = TRIAL_LABELS_DF["tone_stop_frame"].astype(int)


TRIAL_LABELS_DF = TRIAL_LABELS_DF.drop(columns=[col for col in TRIAL_LABELS_DF.columns if "unnamed" in col], errors="ignore")
TRIAL_LABELS_DF = TRIAL_LABELS_DF.drop(columns=[col for col in TRIAL_LABELS_DF.columns if "box" in col], errors="ignore")

In [22]:
TRIAL_LABELS_DF.head()

,video_name,tone_start_frame,reward_start_frame,tone_stop_frame,condition,competition_closeness,notes,experiment,session_dir,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp
1,20230612_101430_standard_comp_to_training_D1_s...,980,1080,1181,1.4,Subj 2 Only,NaN,standard,20230612_101430_standard_comp_to_training_D1_s...,"['1.3', '1.4']",8798886.0,77093151.0,982229,1182226
2,20230612_101430_standard_comp_to_training_D1_s...,3376,3476,3575,1.3,Subj 1 Only,NaN,standard,20230612_101430_standard_comp_to_training_D1_s...,"['1.3', '1.4']",8798886.0,77093151.0,3382227,3582224
3,20230612_101430_standard_comp_to_training_D1_s...,5672,5772,5871,1.4,Subj 2 Only,NaN,standard,20230612_101430_standard_comp_to_training_D1_s...,"['1.3', '1.4']",8798886.0,77093151.0,5682225,5882222
4,20230612_101430_standard_comp_to_training_D1_s...,7468,7568,7668,1.4,Subj 2 Only,NaN,standard,20230612_101430_standard_comp_to_training_D1_s...,"['1.3', '1.4']",8798886.0,77093151.0,7482224,7682221
5,20230612_101430_standard_comp_to_training_D1_s...,8566,8666,8765,1.4,Subj 2 Only,NaN,standard,20230612_101430_standard_comp_to_training_D1_s...,"['1.3', '1.4']",8798886.0,77093151.0,8582220,8782223


In [23]:
merged_df = pd.merge(subject_start_stop_frames, TRIAL_LABELS_DF, on="video_name", how="inner")

In [25]:
merged_df = merged_df.dropna(subset=["current_subject"])

merged_df = merged_df[(merged_df['tone_start_frame'] > merged_df['start_frame']) & (merged_df['tone_start_frame'] < merged_df['stop_frame'])]

In [26]:
merged_df = merged_df.drop(columns=["start_frame", "stop_frame"], errors="ignore")
merged_df = merged_df.drop(columns=["first_timestamp", "last_timestamp"], errors="ignore")
merged_df = merged_df.drop(columns=["box_1_port_entry_frames", "box_2_port_entry_frames"], errors="ignore")
merged_df = merged_df.drop(columns=['box_1_port_entry_timestamps', 'box_2_port_entry_timestamps'], errors="ignore")


In [27]:
merged_df = merged_df.sort_values(by=["session_dir", "current_subject", "tone_start_timestamp"]).reset_index(drop=True)

In [28]:
merged_df.columns

Index(['tracked_subject', 'sleap_name', 'video_name', 'current_subject',
       'tone_start_frame', 'reward_start_frame', 'tone_stop_frame',
       'condition', 'competition_closeness', 'notes', 'experiment',
       'session_dir', 'all_subjects', 'tone_start_timestamp',
       'tone_stop_timestamp'],
      dtype='object')

In [29]:
merged_df.head()

,tracked_subject,sleap_name,video_name,current_subject,tone_start_frame,reward_start_frame,tone_stop_frame,condition,competition_closeness,notes,experiment,session_dir,all_subjects,tone_start_timestamp,tone_stop_timestamp
0,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1125,1225,1324,1.1,Subj 1 blocking Subj 2,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",1126742,1326741
1,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,3519,3619,3720,1.2,Subj 2 Only,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",3526740,3726740
2,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,5815,5915,6014,1.2,Subj 2 blocking Subj 1,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",5826740,6026737
3,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,7612,7712,7811,1.2,Subj 2 Only,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",7626736,7826735
4,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,8709,8809,8910,1.2,Subj 2 Only,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",8726734,8926734


In [25]:
non_competitive_labels = ['rewarded', 'omission', 'both_rewarded']

In [26]:
merged_df["current_subject"] = merged_df["current_subject"].astype(str)
merged_df["condition"] = merged_df["condition"].astype(str)

In [27]:
merged_df["trial_label"] = merged_df.apply(lambda x: x["condition"] if x["condition"] in non_competitive_labels else ("win" if x["condition"] == x["current_subject"] else "lose"), axis=1)

In [28]:
merged_df

,tracked_subject,sleap_name,video_name,current_subject,tone_start_frame,reward_start_frame,tone_stop_frame,condition,competition_closeness,notes,experiment,session_dir,all_subjects,tone_start_timestamp,tone_stop_timestamp,trial_label
0,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1125,1205,1324,1.1,Subj 1 blocking Subj 2,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",1126742,1326741,win
1,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,3519,3599,3720,1.2,Subj 2 Only,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",3526740,3726740,lose
2,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,5815,5895,6014,1.2,Subj 2 blocking Subj 1,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",5826740,6026737,lose
3,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,7612,7692,7811,1.2,Subj 2 Only,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",7626736,7826735,lose
4,1.1_1.2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,8709,8789,8910,1.2,Subj 2 Only,NaN,standard,20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']",8726734,8926734,lose
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824,1.1_1.4,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,1.4,65186,65266,65386,both_rewarded,NaN,NaN,both_rewarded,20230625_112913_standard_comp_to_both_rewarded...,"['1.1', '1.4']",64304011,64504015,both_rewarded
825,1.1_1.4,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,1.4,66684,66764,66883,both_rewarded,NaN,NaN,both_rewarded,20230625_112913_standard_comp_to_both_rewarded...,"['1.1', '1.4']",65804029,66004033,both_rewarded
826,1.1_1.4,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,1.4,68680,68760,68879,both_rewarded,NaN,NaN,both_rewarded,20230625_112913_standard_comp_to_both_rewarded...,"['1.1', '1.4']",67804057,68004058,both_rewarded
827,1.1_1.4,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,1.4,70576,70656,70775,both_rewarded,NaN,NaN,both_rewarded,20230625_112913_standard_comp_to_both_rewarded...,"['1.1', '1.4']",69704075,69904080,both_rewarded


In [29]:
merged_df.to_excel("rce2_finalized_trial_labels.xlsx", index=False)

# TODO
- Merge recording file name
- Make a dictionary of trial labels 
- Add box number to sheet

In [30]:
trodes_metadata_df = pd.read_pickle("./rce_pilot_2_00_trodes_metadata.pkl")
trodes_metadata_df["video_name"] = trodes_metadata_df["video_name"].apply(lambda x: ".".join(os.path.basename(x).split(".")[:2]))

In [31]:
trodes_metadata_df = trodes_metadata_df.drop(columns = ["first_timestamp", "last_timestamp", "tone_timestamps", "tone_frames"], errors="ignore")

In [32]:
trodes_metadata_df = trodes_metadata_df.drop(columns=["all_subjects"], errors="ignore")

In [33]:
trodes_metadata_df.head()

,session_dir,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,video_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,20230612_101430_standard_comp_to_training_D1_s...,"[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [..."
1,20230612_101430_standard_comp_to_training_D1_s...,"[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [..."
2,20230612_101430_standard_comp_to_training_D1_s...,"[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [..."
3,20230612_101430_standard_comp_to_training_D1_s...,"[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [..."
4,20230612_112630_standard_comp_to_training_D1_s...,"[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,"[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[192745, 249350], [389747, 407142], [917544, ...","[[33037711, 33038706], [33264908, 33270313], [..."


In [34]:
final_trials_df = pd.merge(trodes_metadata_df, merged_df, on=["video_name", "current_subject", "session_dir"], how="inner")

In [35]:
final_trials_df

,session_dir,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,video_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps,...,reward_start_frame,tone_stop_frame,condition,competition_closeness,notes,experiment,all_subjects,tone_start_timestamp,tone_stop_timestamp,trial_label
0,20230612_112630_standard_comp_to_training_D1_s...,"[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,"[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[192745, 249350], [389747, 407142], [917544, ...","[[33037711, 33038706], [33264908, 33270313], [...",...,1205,1324,1.1,Subj 1 blocking Subj 2,NaN,standard,"['1.1', '1.2']",1126742,1326741,win
1,20230612_112630_standard_comp_to_training_D1_s...,"[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,"[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[192745, 249350], [389747, 407142], [917544, ...","[[33037711, 33038706], [33264908, 33270313], [...",...,3599,3720,1.2,Subj 2 Only,NaN,standard,"['1.1', '1.2']",3526740,3726740,lose
2,20230612_112630_standard_comp_to_training_D1_s...,"[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,"[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[192745, 249350], [389747, 407142], [917544, ...","[[33037711, 33038706], [33264908, 33270313], [...",...,5895,6014,1.2,Subj 2 blocking Subj 1,NaN,standard,"['1.1', '1.2']",5826740,6026737,lose
3,20230612_112630_standard_comp_to_training_D1_s...,"[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,"[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[192745, 249350], [389747, 407142], [917544, ...","[[33037711, 33038706], [33264908, 33270313], [...",...,7692,7811,1.2,Subj 2 Only,NaN,standard,"['1.1', '1.2']",7626736,7826735,lose
4,20230612_112630_standard_comp_to_training_D1_s...,"[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,"[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[192745, 249350], [389747, 407142], [917544, ...","[[33037711, 33038706], [33264908, 33270313], [...",...,8789,8910,1.2,Subj 2 Only,NaN,standard,"['1.1', '1.2']",8726734,8926734,lose
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744,20230625_112913_standard_comp_to_both_rewarded...,"[[308, 312], [1165, 1194], [1557, 1571], [1643...",[],20230625_112913_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230625_112913_standard_comp_to_both_rewarded...,1.4,"[1384, 2770, 2770, 4156, 5542, 6928, 6928, 831...","[[310252, 314054], [1167262, 1196060], [156166...",[],...,65266,65386,both_rewarded,NaN,NaN,both_rewarded,"['1.1', '1.4']",64304011,64504015,both_rewarded
745,20230625_112913_standard_comp_to_both_rewarded...,"[[308, 312], [1165, 1194], [1557, 1571], [1643...",[],20230625_112913_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230625_112913_standard_comp_to_both_rewarded...,1.4,"[1384, 2770, 2770, 4156, 5542, 6928, 6928, 831...","[[310252, 314054], [1167262, 11

In [36]:
final_trials_df["tone_timestamps"] = final_trials_df.apply(lambda x: [x["tone_start_timestamp"], x["tone_stop_timestamp"]], axis=1)

In [37]:
final_trials_df["tone_frames"] = final_trials_df.apply(lambda x: [x["tone_start_frame"], x["tone_stop_frame"]], axis=1)

In [38]:
final_trials_df = final_trials_df.drop(columns=["tone_start_timestamp", "tone_stop_timestamp", "condition", "tone_start_frame", "tone_stop_frame", "tracked_subject", "reward_start_frame", "notes"], errors="ignore")

In [39]:
grouping_columns = ["session_dir", "current_subject", "trial_label", "experiment"]

In [40]:
non_grouping_columns = set(final_trials_df.columns) - set(grouping_columns)

In [41]:
final_trials_df["tone_timestamps"].iloc[0]

[1126742, 1326741]

In [42]:
grouped_finals_trials_df = final_trials_df.groupby(grouping_columns).agg({
    col: (lambda col: list(col)) if col in ['tone_timestamps', 'tone_frames'] else 'first' for col in non_grouping_columns}).reset_index()

In [43]:
grouped_finals_trials_df["tone_frames"] = grouped_finals_trials_df["tone_frames"].apply(lambda x: np.array(x))
grouped_finals_trials_df["tone_timestamps"] = grouped_finals_trials_df["tone_timestamps"].apply(lambda x: np.array(x))

In [44]:
grouped_finals_trials_df = grouped_finals_trials_df.drop(columns=["competition_closeness"], errors="ignore")   

In [45]:
grouped_finals_trials_df.head()

,session_dir,current_subject,trial_label,experiment,recording,sleap_name,tone_timestamps,video_name,all_subjects,video_timestamps,box_2_port_entry_timestamps,box_2_port_entry_frames,tone_frames,session_path,box_1_port_entry_timestamps,box_1_port_entry_frames
0,20230612_112630_standard_comp_to_training_D1_s...,1.1,lose,standard,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[3526740, 3726740], [5826740, 6026737], [7626...",20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']","[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[33037711, 33038706], [33264908, 33270313], [...","[[33019, 33020], [33246, 33251], [33253, 33255...","[[3519, 3720], [5815, 6014], [7612, 7811], [87...",/scratch/back_up/reward_competition_extention/...,"[[192745, 249350], [389747, 407142], [917544, ...","[[192, 248], [389, 405], [916, 929], [929, 948..."
1,20230612_112630_standard_comp_to_training_D1_s...,1.1,rewarded,standard,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[37126704, 37326704], [38326703, 38526703], [...",20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']","[1384, 2531, 2769, 4155, 5541, 6799, 6927, 831...","[[33037711, 33038706], [33264908, 33270313], [...","[[32974, 32976], [33201, 33207], [33208, 33211...","[[37056, 37256], [38254, 38453], [39352, 39552...",/scratch/back_up/reward_competition_extention/...,"[[192745, 249350], [389747, 407142], [917544, ...","[[192, 248], [389, 405], [916, 930], [930, 948..."
2,20230612_112630_standard_comp_to_training_D1_s...,1.1,win,standard,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1126742, 1326741], [22826720, 23026720], [24...",20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']","[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[33037711, 33038706], [33264908, 33270313], [...","[[33019, 33020], [33246, 33251], [33253, 33255...","[[1125, 1324], [22784, 22983], [24680, 24879],...",/scratch/back_up/reward_competition_extention/...,"[[192745, 249350], [389747, 407142], [917544, ...","[[192, 248], [389, 405], [916, 929], [929, 948..."
3,20230612_112630_standard_comp_to_training_D1_s...,1.2,lose,standard,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1126742, 1326741], [22826720, 23026720], [24...",20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']","[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[33037711, 33038706], [33264908, 33270313], [...","[[33019, 33020], [33246, 33251], [33253, 33255...","[[1125, 1324], [22784, 22983], [24680, 24879],...",/scratch/back_up/reward_competition_extention/...,"[[192745, 249350], [389747, 407142], [917544, ...","[[192, 248], [389, 405], [916, 929], [929, 948..."
4,20230612_112630_standard_comp_to_training_D1_s...,1.2,rewarded,standard,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[37126704, 37326704], [38326703, 38526703], [...",20230612_112630_standard_comp_to_training_D1_s...,"['1.1', '1.2']","[1384, 2444, 2769, 4155, 5541, 6708, 6927, 831...","[[33037711, 33038706], [33264908, 33270313], [...","[[33019, 33020], [33246, 33251], [33253, 33255...","[[37101, 37300], [38299, 38498], [39397, 39597...",/scratch/back_up/reward_competition_extention/...,"[[192745, 249350], [389747, 407142], [917544, ...","[[192, 248], [389, 405], [916, 929], [929, 948..."


In [46]:
grouped_finals_trials_df["tone_timestamps"] = grouped_finals_trials_df["tone_timestamps"].apply(lambda x: x // 20)

In [47]:
grouped_finals_trials_df["trial_label"].unique()

array(['lose', 'rewarded', 'win', 'omission', 'both_rewarded'],
      dtype=object)

In [48]:
dict_from_df = {k1: {k2: v for k2, v in zip(df['trial_label'], df['tone_timestamps'])} for k1, df in grouped_finals_trials_df.groupby('recording')}

In [49]:
dict_from_df

{'20230612_112630_standard_comp_to_training_D1_subj_1-1_t1b3L_box2_merged': {'lose': array([[ 176337,  186337],
         [ 291337,  301336],
         [ 381336,  391336],
         [ 436336,  446336],
         [ 486336,  496336],
         [ 581336,  591336],
         [ 656336,  666336],
         [ 761336,  771336],
         [ 811336,  821336],
         [ 891336,  901336],
         [ 956336,  966336],
         [1021336, 1031336],
         [1071336, 1081336]]),
  'rewarded': array([[1856335, 1866335],
         [1916335, 1926335],
         [1971335, 1981335],
         [2036335, 2046335],
         [2091334, 2101334],
         [2141334, 2151334],
         [2191334, 2201334],
         [2296334, 2306334],
         [2416334, 2426334],
         [2536334, 2546334],
         [2646334, 2656334],
         [2731334, 2741334],
         [2851334, 2861334],
         [2911334, 2921334],
         [2976334, 2986333],
         [3036333, 3046333],
         [3111333, 3121333],
         [3211333, 3221333],
    

In [50]:
import pickle


In [51]:
            
with open('recording_to_label_to_timestamps.pkl', 'wb') as f:
    pickle.dump(dict_from_df, f)

In [52]:
merged_df.to_pickle("rce2_finalized_trial_labels.pkl")

In [53]:
grouped_finals_trials_df.to_pickle("rce2_grouped_finalized_trial_labels.pkl")